# Data Cleaning
This notebook will:
* Ingest raw data
* Impute missing values
* Flag correlated variables (retaining all fields to selectively drop later)
* Create binary flags for “zero-heavy” columns (retaining all fields)
* Standardise categorical strings
* Output a cleaned CSV in data/processed/
* Serve as the basis for the contents of src/data_cleaning.py

In [8]:
# notebooks/01_data_cleaning.ipynb

# ---
# HR Attrition Dataset - Data Cleaning
# ---

# Step 1. Imports
import pandas as pd
import numpy as np
from pathlib import Path

# Step 2. Paths
DATA_DIR = Path("../data")
RAW_PATH = DATA_DIR / "raw" / "hr_attrition_dataset.csv"
PROCESSED_PATH = DATA_DIR / "processed" / "hr_attrition_clean.csv"

# Step 3. Load raw dataset
df = pd.read_csv(RAW_PATH)
print("Raw dataset shape:", df.shape)
df.head()

Raw dataset shape: (25000, 37)


,employee_id,snapshot_date,hire_date,region,department,role,level,is_manager,age,gender,...,benefit_score,sick_days,pto_days_taken,leave_last_yr,team_id,exit_interview_scheduled,offboarding_ticket_created,attrited,months_since_hire,salary_band
0,121752,2020-05-13,2014-08-10,AMER,Sales,Sales Manager,IC3,0,33,F,...,6.99,9.1,16.7,0,85,0,0,0,69.1,Q4
1,107208,2020-09-20,2011-07-14,APAC,Engineering,SWE II,Manager,1,31,F,...,7.75,8.4,15.6,1,6,0,0,0,110.2,Q5
2,117058,2020-12-29,2013-07-28,EMEA,Sales,AE,IC2,0,40,M,...,6.76,6.6,19.5,0,177,0,0,0,89.1,Q3
3,120754,2023-10-31,2012-04-06,AMER,Engineering,ML Engineer,IC4,0,26,M,...,7.84,9.4,18.8,0,232,0,0,0,138.8,Q4
4,122505,2022-07-20,2015-05-25,APAC,HR,HRBP,IC3,0,31,F,...,6.05,5.4,17.4,0,328,1,0,1,85.8,Q3


In [ ]:
# Step 4. Handle missing values

# engagement_score & manager_quality have missing values (~7-8%)
# Impute with the median for their respective columns
for col in ["engagement_score", "manager_quality"]:
    if col in df.columns:
        min_val = df[col].median()
        df[col].fillna(min_val, inplace=True)
        print(f"Filled missing values in {col} with minimum {min_val:.2f}")

Filled missing values in engagement_score with minimum 7.29
Filled missing values in manager_quality with minimum 7.09


/var/folders/79/x31jymnx6cq2mjpqsmf1x02r0000gn/T/ipykernel_72866/2047983168.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(min_val, inplace=True)
/var/folders/79/x31jymnx6cq2mjpqsmf1x02r0000gn/T/ipykernel_72866/2047983168.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

In [ ]:
# Step 5. Flag highly correlated features

highly_corr = {
    "exit_interview_scheduled": "attrited",
    "offboarding_ticket_created": "attrited",
    "months_since_hire": "tenure_years",
    "role": "department",
    "salary_band": "base_salary",
    "stock_grants": "base_salary"
}
print("Highly correlated field pairs flagged:", highly_corr)

Highly correlated field pairs flagged: {'exit_interview_scheduled': 'attrited', 'offboarding_ticket_created': 'attrited', 'months_since_hire': 'tenure_years', 'role': 'department', 'salary_band': 'base_salary', 'stock_grants': 'base_salary'}


In [11]:
# Step 6. Handle zeros / imbalance

# Commute_km has 35% zeros — likely "walks/bikes/remote" → flag instead of dropping
if "commute_km" in df.columns:
    df["commute_zero"] = (df["commute_km"] == 0).astype(int)

# Internal_moves_last_2y has 78% zeros — convert to binary flag
if "internal_moves_last_2y" in df.columns:
    df["internal_moves_flag"] = (df["internal_moves_last_2y"] > 0).astype(int)

# Stock grants 68% zeros — flag them separately
if "stock_grants" in df.columns:
    df["stock_grants_flag"] = (df["stock_grants"] > 0).astype(int)

In [ ]:
# Step 7. Encode categorical variables (basic handling for now)

cat_cols = df.select_dtypes(include=["object"]).columns
print("Categorical columns:", list(cat_cols))

# Will one-hot encode later in modelling.
# But strip whitespace and lowercase to avoid duplication.
for col in cat_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()

Categorical columns: ['snapshot_date', 'hire_date', 'region', 'department', 'role', 'level', 'gender', 'remote_status', 'salary_band']


In [13]:
# Step 8. Save cleaned dataset
df.to_csv(PROCESSED_PATH, index=False)
print("Saved cleaned dataset to:", PROCESSED_PATH)
print("Final shape:", df.shape)

Saved cleaned dataset to: ../data/processed/hr_attrition_clean.csv
Final shape: (25000, 40)
